In [113]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

a = 3
s = 3
r = 3
rr = 20
ro = 70
ra = 300
noise = 0.01
prop = 0.4
weight = 3
bias = np.array([0.0,-1.0])
dev_bias = 3
dTheta = 120
sight_theta = 180


In [163]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    seed()

    agents = [np.array([0.0, 0.0]) for i in range(N)]
    #agents = [Point(uniform(0, width), uniform(0, height)) for i in range(N)]
    
    speeds = N*[0,0]
    #speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = uniform(0, 2 * np.pi)
        #speeds[i][0] = speed * np.cos(theta)
        #speeds[i][1] = speed * np.sin(theta)
        speeds[i]= speed * np.array([np.cos(theta),np.sin(theta)])
        
        theta = uniform(0, 2 * np.pi)
        radius = uniform(0,height/2)
        agents[i] = Point(width/2 + radius*np.cos(theta), height/2 + radius*np.sin(theta))

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2))) # radians
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        a, dists[i] = relative_pos(agent, agents[i])
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    #return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repulsion radius.")
        

        
def virtualizer (current, agents, h, w, N):
    lower_limit = current.getY() - h / 2
    upper_limit = current.getY() + h / 2
    left_limit = current.getX() - w / 2
    right_limit = current.getX() + w / 2
    
    vagents=[np.array([agent.getX(),agent.getY()]) for agent in agents]
    virtuals = N * [0.0, 0.0]
    vvirtuals = N * [Point(0.0, 0.0)]

    for j in range(N):
        #make more compact!!!
        virtuals[j]=vagents[j]
        
        #newcopy = agents[i].clone()
        candidates = [vagents[j],
                      vagents[j]+[w,0],vagents[j]+[-w,0],vagents[j]+[w,h],vagents[j]+[w,-h],
                      vagents[j]+[-w,h],vagents[j]+[-w,-h],vagents[j]+[0,h],vagents[j]+[0,-h]]
        #print candidates
        
        #virtuals[j] = next((cand for cand in candidates if lower_limit < cand[1] < upper_limit and left_limit < cand[0] < right_limit),False
        for i in range(9):
            if lower_limit < candidates[i][1] < upper_limit and left_limit < candidates[i][0] < right_limit:
                virtuals[j]=candidates[i]
            
        vvirtuals[j] = Point(virtuals[j][0],virtuals[j][1])
        
    return vvirtuals


def get_cm_std(agents, N):
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    return np.mean(poses,axis = 0), np.std(poses,axis = 0)

def mill_observables (N, agents, speeds):
    cm, std = get_cm_std(agents, N)
    mean_R = np.linalg.norm(std)
    point_cm = Point(cm[0],cm[1])
    norm_R = N*[0.0]
    vector_R = N*[0,0]
    versor_R = N*[0,0]
    versor_T = N*[0,0]
    
    proj_radial = N*[0]
    radial = N*[0,0]
    
    proj_tangencial= N*[0]
    norm_tangencial= N*[0]
    tangencial= N*[0,0]
    
    angles = N*[0]
    angular= N*[0,0]
    for i in range(N):
        vector_R[i],norm_R[i] = relative_pos(point_cm, agents[i])
        
        versor_R[i] = normalized(vector_R[i])
        proj_radial[i] = np.dot(speeds[i], versor_R[i])
        radial[i] = proj_radial[i]*versor_R[i]
        
        versor_T[i] = versor_R[i]*rot_matrix(90,'deg')
        norm_tangencial[i]= norm_R[i]*np.linalg.norm(angular[i])
        proj_tangencial[i] = np.dot(speeds[i], versor_T[i])
        tangencial[i] = proj_tangencial[i]*versor_T[i]
        
        angular[i] = np.cross(np.array(vector_R[i]),np.array(speeds[i]))
        angles[i] = (np.pi - angle(vector_R[i],speeds[i]))*360/(2*np.pi)
        
    print proj_radial[0], radial[0]
    print proj_tangencial[0], norm_tangencial[0], tangencial[0]
    print angular[0]
    print angles[0]
    
    
    min_R = min(norm_R)
    max_R = max(norm_R)
    
    #return mean_R, min_R, max_R, 
    #return angles



In [ ]:

##MILL MODEL
def mill (dt, agents, speeds, N, width, height, cr, ca, lr, la, alpha, beta, mass):
    clr = cr / lr
    cla = ca / la
    for i in range(N):
        u_dir = np.array([0.0, 0.0])
        propulsion = np.array([0.0, 0.0])
        friction = np.array([0.0, 0.0])
        grad_U = np.array([0.0, 0.0])
        
        #virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):   #Duality interactions, by the Morse potential  
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], agents[j])
            u_dir = normalized(rel_pos)
            grad_U = grad_U + u_dir * (clr*np.exp(-distance / lr) - cla *np.exp(- distance / la))
            
        
        propulsion = alpha * speeds[i] # self-propulsion propto alpha
        norm = (np.linalg.norm(speeds[i]))
        friction =  beta *((norm)**2) * speeds[i] #friction force propto beta
        
        #print i,  agents[i]
        #print speeds[i], norm
        #print propulsion, friction, grad_U
        
        
        d_speed = (propulsion - friction - grad_U)/mass
        speeds[i]= speeds[i]+dt*d_speed
        #print u_dir
    
   # print speeds[N-1], propulsion, friction, grad_U
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    bias = np.array([0.0,-1.0])
    for i in range(N_steps):
        
        #print 'step', i
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_theta, 1, 200, 1.5, 1) #(...)last 4 parameters: model2, roa, orient, atract
        
        mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=120, la=250.0, alpha=1.0, beta=0.01, mass=1) 
        # Region II, really nice ring
        #mill (dt, agents, speeds, N, width, height, cr=110, ca=100, lr=20, la=100.0, alpha=4.0, beta=0.06, mass=3) 
        # Region VII,(l<1/sqrt(C)) alpha ~ beta, best vortex in this region, decreasing in size with increasing N
        
        mill_observables (N, agents, speeds)
        print i
        #print angles
        
        
        #interaction(agents, speeds)
        #biaser(speeds, N, s, i, prop, bias, dev_bias, weight)
        
        ## BOUNDARY CONDITIONS
        rigid_boundary(width, height, agents, speeds, N)
        #periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return

In [ ]:
run(100000, 50, 5, 0.3, 900, 700)

3.60946069139 [ 2.46147913  2.63994829]
[ 0.74436725 -3.01494326] 0.0 [[  3.10829302e-17   2.20512010e+00]
 [  5.07622774e-01  -1.35024664e-16]]
-615.87865188
147.827545393
0
2.38963804029 [ 1.63589424  1.74190126]
[-0.16197194 -2.728132  ] 0.0 [[ -6.78959298e-18   1.98864283e+00]
 [ -1.10882468e-01  -1.21769254e-16]]
-788.123124592
129.585121274
1
0.894090183014 [ 0.61496342  0.64901252]
[-1.30550582 -2.39766753] 0.0 [[ -5.49828974e-17   1.74044663e+00]
 [ -8.97938857e-01  -1.06571620e-16]]
-1014.11276481
103.573661835
2
-0.99451921032 [-0.6879326  -0.71820414]
[-2.76048179 -1.97048779] 0.0 [[ -1.16922599e-16   1.42301173e+00]
 [ -1.90949094e+00  -8.71343379e-17]]
-1299.08515859
78.1284412064
3
-3.34570418524 [-2.32992167 -2.4010834 ]
[-4.53962621 -1.37327393] 0.0 [[ -1.93577435e-16   9.85545958e-01]
 [ -3.16135943e+00  -6.03472852e-17]]
-1624.70298874
60.4974584834
4
-6.01239189225 [-4.21985222 -4.28272151]
[-6.44697385 -0.53853752] 0.0 [[ -2.77068227e-16   3.83608761e-01]
 [ -4.5248

-13.2250572686 [-10.17348619  -8.44998927]
[-6.61121037  6.42365871] 0.0 [[ -3.11411087e-16  -4.10431850e+00]
 [ -5.08572901e+00   2.51317025e-16]]
-22.4018324915
0.81248782528
48
-13.2156930835 [-10.17174771  -8.43742213]
[-6.59899427  6.42183628] 0.0 [[ -3.11002760e-16  -4.09995475e+00]
 [ -5.07906051e+00   2.51049823e-16]]
-20.4575593314
0.768010957839
49
-13.2034847184 [-10.16800706  -8.42280483]
[-6.58532965  6.41852297] 0.0 [[ -3.10531491e-16  -4.09452258e+00]
 [ -5.07136410e+00   2.50717198e-16]]
-18.5991933705
0.723809700946
50
-13.1882500415 [-10.16147915  -8.40679966]
[-6.56984456  6.41411892] 0.0 [[ -3.09960040e-16  -4.08865563e+00]
 [ -5.06203160e+00   2.50357952e-16]]
-16.7440073784
0.676511818075
51
-13.1697922081 [-10.15135466  -8.39007898]
[-6.55223834  6.40898091] 0.0 [[ -3.09254210e-16  -4.08296920e+00]
 [ -5.05050453e+00   2.50009758e-16]]
-14.833335346
0.623223377915
52
-13.1478693585 [-10.1369984   -8.37303601]
[-6.53234835  6.4032466 ] 0.0 [[ -3.08392772e-16  -4.0

-4.22250605795 [ 2.49279439  3.40815691]
[ 1.39547349  8.3815065 ] 0.0 [[ -5.04450580e-17   6.76505585e+00]
 [ -8.23830316e-01  -4.14240200e-16]]
367.096088523
66.6413317102
96
-5.26878799182 [ 2.8675505   4.42009966]
[ 0.4319433   9.14770186] 0.0 [[ -1.43948766e-17   7.67420401e+00]
 [ -2.35086176e-01  -4.69909469e-16]]
350.913574579
61.1892836109
97
-6.13951925562 [ 3.04739009  5.32983211]
[-0.36202374  9.6305032 ] 0.0 [[  1.10030120e-17   8.36042091e+00]
 [  1.79692823e-01  -5.11928136e-16]]
327.600485841
56.4791717311
98
-6.8384047025 [ 3.05697739  6.11708003]
[-0.95442517  9.86219587] 0.0 [[  2.61252328e-17   8.82191740e+00]
 [  4.26657430e-01  -5.40186645e-16]]
300.55321488
52.486918404
99
-7.37570678458 [ 2.92237427  6.77205871]
[-1.34170784  9.88693236] 0.0 [[  3.25515000e-17   9.07775869e+00]
 [  5.31606338e-01  -5.55852406e-16]]
272.291638259
49.2012800925
100
-7.76294863055 [ 2.66626788  7.29070552]
[-1.5349924   9.74994068] 0.0 [[  3.22822766e-17   9.15682297e+00]
 [  5.272

-2.89140655074 [ 2.84468285 -0.51769808]
[ 10.16964914  -0.18938627] 0.0 [[ -6.12648705e-16   3.39090713e-02]
 [ -1.00053126e+01  -2.07633178e-18]]
186.995588495
73.8430386965
144
-2.02516445995 [ 2.02511637 -0.01395676]
[ 10.08340181  -0.01347815] 0.0 [[ -6.17415625e-16   9.28869258e-05]
 [ -1.00831623e+01  -5.68768382e-21]]
184.398238359
78.6289168122
145
-1.17690532055 [ 1.16076165  0.19426406]
[ 9.61878097  0.46637905] 0.0 [[ -5.80901391e-16   7.69821386e-02]
 [ -9.48683966e+00  -4.71379648e-18]]
183.498754528
83.3438748869
146
-0.409153265108 [ 0.38664359  0.13383993]
[ 8.83207407  1.19994078] 0.0 [[ -5.11055839e-16   3.92517923e-01]
 [ -8.34617524e+00  -2.40347909e-17]]
184.141528423
87.6645000673
147
0.233176226733 [-0.20554463 -0.11010248]
[ 7.81450997  2.11734434] 0.0 [[ -4.21797966e-16   9.99779706e-01]
 [ -6.88848353e+00  -6.12188509e-17]]
186.129339607
91.3449210391
148
0.728771663543 [-0.58471999 -0.43498353]
[ 6.66511367  3.14641782] 0.0 [[ -3.27449912e-16   1.87800928e+0

-0.852485282606 [ 0.72365892 -0.4506095 ]
[ 7.6331253   2.42878768] 0.0 [[ -3.96762191e-16  -1.28381664e+00]
 [ -6.47961830e+00   7.86110972e-17]]
201.716411063
85.1572389617
192
-0.236530717967 [ 0.21823591 -0.09121331]
[ 8.59987132  1.40343768] 0.0 [[ -4.85860369e-16  -5.41207476e-01]
 [ -7.93470199e+00   3.31394002e-17]]
197.824416604
88.6454795078
193
0.386337165592 [-0.37590937  0.08915466]
[ 9.28639538  0.61398673] 0.0 [[ -5.53279634e-16  -1.41689138e-01]
 [ -9.03574213e+00   8.67595746e-18]]
195.477932146
92.2346878577
194
0.970266847324 [-0.96779825  0.06916862]
[ 9.6400921   0.11858638] 0.0 [[ -5.88783567e-16  -8.45381555e-03]
 [ -9.61556535e+00   5.17646908e-19]]
194.784721426
95.6780319294
195
1.47427289271 [-1.46890992 -0.12563524]
[ 9.6487467  -0.05551038] 0.0 [[ -5.88666124e-16  -4.73050802e-03]
 [ -9.61364738e+00   2.89660075e-19]]
195.797797986
98.736770829
196
1.86849240337 [-1.81733231 -0.43424318]
[ 9.3376764   0.08666605] 0.0 [[ -5.56112538e-16   2.01414472e-02]
 [ 

...

-0.613626880764 [-0.37116956  0.48864211]
[ 3.94675434  6.03249638] 0.0 [[  1.46180325e-16   4.80378522e+00]
 [  2.38730588e+00  -2.94147009e-16]]
244.004033167
86.4812970447
4297
-0.614692508814 [-0.42893823  0.44029408]
[ 5.16285084  4.80886932] 0.0 [[  2.20600894e-16   3.44451357e+00]
 [  3.60268600e+00  -2.10915626e-16]]
243.845138951
86.4725466577
4298
-0.62091100749 [-0.48442904  0.38840569]
[ 6.37098022  3.59776153] 0.0 [[  3.04360245e-16   2.25054967e+00]
 [  4.97058002e+00  -1.37806422e-16]]
243.784999851
86.4358910624
4299
-0.630655196014 [-0.53658974  0.33135696]
[ 7.49978999  2.47049304] 0.0 [[  3.90733223e-16   1.29803902e+00]
 [  6.38115780e+00  -7.94819666e-17]]
243.810745353
86.380663845
4300
-0.641590708875 [-0.58304233  0.26776907]
[ 8.48126437  1.494222  ] 0.0 [[  4.71936406e-16   6.23616314e-01]
 [  7.70730640e+00  -3.81854861e-17]]
243.894491117
86.3199912661
4301
-0.651172083791 [-0.62058079  0.19724241]
[ 9.25498458  0.72796578] 0.0 [[  5.40081260e-16   2.2050350

-0.63969048243 [-0.09359659  0.63280613]
[ 0.30665749  9.69269534] 0.0 [[  2.74741689e-18   9.58838242e+00]
 [  4.48687229e-02  -5.87119092e-16]]
244.501203315
86.3395943523
4344
-0.636023435308 [-0.16954294  0.61300979]
[ 0.8740715   9.12709475] 0.0 [[  1.42670577e-17   8.79684319e+00]
 [  2.32998733e-01  -5.38651292e-16]]
244.52203502
86.3611793309
4345
-0.628963704916 [-0.24077551  0.58105292]
[ 1.68768428  8.31089249] 0.0 [[  3.95602276e-17   7.67781722e+00]
 [  6.46067545e-01  -4.70130714e-16]]
244.442281707
86.4005332547
4346
-0.621073630439 [-0.30634245  0.54026545]
[ 2.69724411  7.29386978] 0.0 [[  8.14638999e-17   6.34486100e+00]
 [  1.33040645e+00  -3.88510686e-16]]
244.257732657
86.442922323
4347
-0.615726629583 [-0.36706925  0.49434749]
[ 3.84207658  6.13920091] 0.0 [[  1.40251316e-16   4.92897081e+00]
 [  2.29047782e+00  -3.01812416e-16]]
244.029216811
86.4700020463
4348
-0.615214035875 [-0.42449101  0.44530404]
[ 5.05529821  4.91780086] 0.0 [[  2.13584583e-16   3.55960116

-0.632433690636 [ 0.06934129  0.62862084]
[ 0.05110877  9.93378113] 0.0 [[ -3.43125650e-19   9.87389180e+00]
 [ -5.60366712e-03  -6.04601499e-16]]
244.204188829
86.3757797445
4393
-0.639458721841 [-0.00827525  0.63940517]
[  9.94802437e-03   9.98265742e+00] 0.0 [[  7.88290856e-21   9.98182148e+00]
 [  1.28737666e-04  -6.11210286e-16]]
244.388507964
86.3384528848
4394
-0.641802273128 [-0.08690914  0.63589068]
[ 0.2694503   9.72900089] 0.0 [[  2.23420877e-18   9.63938780e+00]
 [  3.64873982e-02  -5.90242271e-16]]
244.506283082
86.3272131146
4395
-0.639236689207 [-0.16359158  0.6179493 ]
[ 0.81316039  9.18807903] 0.0 [[  1.27425493e-17   8.88210442e+00]
 [  2.08101622e-01  -5.43872038e-16]]
244.540890365
86.3428720955
4396
-0.632690636337 [-0.23573739  0.58713314]
[ 1.60699255  8.39271209] 0.0 [[  3.66633202e-17   7.78838678e+00]
 [  5.98757457e-01  -4.76901147e-16]]
244.473948891
86.3796684431
4397
-0.624516366006 [-0.30203746  0.54662059]
[ 2.60177107  7.39133079] 0.0 [[  7.70489845e-17

-0.619798184947 [ 0.14944445  0.60151155]
[ 0.43487599  9.53988568] 0.0 [[ -6.42060223e-18   9.25841924e+00]
 [ -1.04856392e-01  -5.66914675e-16]]
243.959951548
86.4444040222
4443
-0.631323160606 [ 0.07609934  0.62671989]
[ 0.06950153  9.91318488] 0.0 [[ -5.12984653e-19   9.84090329e+00]
 [ -8.37767515e-03  -6.02581536e-16]]
244.183267735
86.3813303669
4444
-0.639752132007 [-0.00123024  0.63975095]
[  1.26718663e-03   9.98948609e+00] 0.0 [[  1.49211011e-22   9.98946762e+00]
 [  2.43680073e-06  -6.11678477e-16]]
244.376016447
86.3361001221
4445
-0.643559067069 [-0.0800901   0.63855607]
[ 0.23430087  9.76302383] 0.0 [[  1.78543960e-18   9.68712649e+00]
 [  2.91584415e-02  -5.93165422e-16]]
244.506850543
86.3167734547
4446
-0.642209785101 [-0.1574601  0.6226072]
[ 0.75387459  9.24720207] 0.0 [[  1.13181010e-17   8.96494373e+00]
 [  1.84838616e-01  -5.48944482e-16]]
244.555971134
86.3258496136
4447
-0.636338739352 [-0.23053149  0.59311232]
[ 1.52740759  8.47322488] 0.0 [[  3.38826753e-17  

-0.604497948387 [ 0.2225937   0.56202297]
[ 1.14451231  8.82258923] 0.0 [[ -2.58059211e-17   8.20267096e+00]
 [ -4.21442674e-01  -5.02268737e-16]]
243.725922848
86.5293011392
4493
-0.617317125844 [ 0.15540717  0.59743539]
[ 0.48162631  9.49103518] 0.0 [[ -7.42427053e-18   9.18536043e+00]
 [ -1.21247539e-01  -5.62441112e-16]]
243.935048131
86.457856918
4494
-0.629823870968 [ 0.08279915  0.6243576 ]
[ 0.09040866  9.89000171] 0.0 [[ -7.27775826e-19   9.80416588e+00]
 [ -1.18854812e-02  -6.00332018e-16]]
244.15875723
86.3890802536
4495
-0.639623909795 [ 0.0058449  0.6395972]
[ -5.01055953e-03   9.99375154e+00] 0.0 [[  2.80362119e-21   9.99333427e+00]
 [  4.57866087e-05  -6.11915242e-16]]
244.359036203
86.3360964992
4496
-0.644931008833 [-0.07314988  0.64076915]
[ 0.20127342  9.79470465] 0.0 [[  1.39787288e-18   9.73149758e+00]
 [  2.28289966e-02  -5.95882368e-16]]
244.50274165
86.3084473509
4497
-0.644901225897 [-0.15114817  0.62693845]
[ 0.69628759  9.3043865 ] 0.0 [[  9.99261626e-18   9.

KeyboardInterrupt: 